# Testing



In [1]:
import pandas as pd
import numpy as np
import math
from bson import ObjectId
from pymongo import MongoClient


import re
import os

import json

client = MongoClient()


In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ordovas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Now we load the dataset and examine how it is loaded.

In [3]:
db = client.get_database("amazon")
# Define an alias for the books dataset
books =db.books
# Obtain all the dataset
res = list(books.find({}))
# Converting to pandas DataFrame
df=pd.DataFrame(res)

In [4]:
df

,_id,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,6033ab6ff7ccad4a92f14abb,5,False,"03 30, 2005",A1REUF3A1YCPHM,0001713353,{'Format:': ' Hardcover'},TW Ervin II,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,1112140800,NaN,NaN
1,6033ab6ff7ccad4a92f14abc,5,True,"06 20, 2016",AVP0HXC9FG790,0001713353,NaN,Amazon Customer,The kids loved it!,Five Stars,1466380800,NaN,NaN
2,6033ab6ff7ccad4a92f14abd,5,True,"01 24, 2016",A324TTUBKTN73A,0001713353,{'Format:': ' Paperback'},Tekla Borner,My students (3 & 4 year olds) loved this book!...,Five Stars,1453593600,NaN,NaN
3,6033ab6ff7ccad4a92f14abe,5,False,"07 9, 2015",A2RE7WG349NV5D,0001713353,{'Format:': ' Paperback'},Deborah K Woroniecki,LOVE IT,Five Stars,1436400000,NaN,NaN
4,6033ab6ff7ccad4a92f14abf,5,True,"01 18, 2015",A32B7QIUDQCD0E,0001713353,NaN,E,Great!,Five Stars,1421539200,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
579820,6033ab9df7ccad4a92fa23a7,1,True,"11 22, 2016",A2MYISBUDCCNHR,0060188731,{'Format:': ' Paperback'},Jules,I had to read this book for book club. I read...,Don't bother...,1479772800,5,NaN
579821,6033ab9df7ccad4a92fa23a8,5,True,"11 21, 2016",AO05IZJ49YDHT,0060188731,{'Format:': ' Kindle Edition'},Mary at Home,"Beautiful, exquisite, but not precious. The l...",Magnificent,1479686400,NaN,NaN
579822,6033ab9df7ccad4a92fa23a9,5,False,"11 19, 2016",AW943ND9EWACD,0060188731,{'Format:': ' Kindle Edition'},Stella,LOVED this book!,Five Stars,1479513600,NaN,NaN
579823,6033ab9df7ccad4a92fa23aa,4,True,"06 20, 2003",ATJDQ137MA426,0060188790,{'Format:': ' Hardcover'},L. Mitchell,"Well, I don't know about the vitamins and the ...",It works!,1056067200,7,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579825 entries, 0 to 579824
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             579825 non-null  object
 1   overall         579825 non-null  int64 
 2   verified        579825 non-null  bool  
 3   reviewTime      579825 non-null  object
 4   reviewerID      579825 non-null  object
 5   asin            579825 non-null  object
 6   style           552457 non-null  object
 7   reviewerName    579803 non-null  object
 8   reviewText      579741 non-null  object
 9   summary         579756 non-null  object
 10  unixReviewTime  579825 non-null  int64 
 11  vote            109253 non-null  object
 12  image           902 non-null     object
dtypes: bool(1), int64(2), object(10)
memory usage: 53.6+ MB


In [6]:
df=df.dropna(subset=['reviewText', 'reviewerName',"summary"])

In [7]:
df["vote"].value_counts()
df=df[df["overall"]!=3] #use only good and bad ones

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526931 entries, 0 to 579824
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             526931 non-null  object
 1   overall         526931 non-null  int64 
 2   verified        526931 non-null  bool  
 3   reviewTime      526931 non-null  object
 4   reviewerID      526931 non-null  object
 5   asin            526931 non-null  object
 6   style           501868 non-null  object
 7   reviewerName    526931 non-null  object
 8   reviewText      526931 non-null  object
 9   summary         526931 non-null  object
 10  unixReviewTime  526931 non-null  int64 
 11  vote            95868 non-null   object
 12  image           826 non-null     object
dtypes: bool(1), int64(2), object(10)
memory usage: 52.8+ MB


In [9]:
df["vote"]=df["vote"].fillna(value=0)
df["vote"]=df["vote"].apply(lambda x: int(str(x).replace(",","")))


In [10]:
len(df[df["vote"]>1]["vote"]),len(df[df["vote"]<1]["vote"])

(95868, 431063)

In [22]:
dff=df.sample(frac=0.01, replace=True, random_state=1)
dff=dff.reset_index()

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(dff['reviewText']).toarray()

In [24]:
tfidf

0       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
5264    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5265    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5266    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5267    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
5268    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Length: 5269, dtype: object

In [31]:
df_tfidf = pd.DataFrame(x, columns=v.get_feature_names())
df2 = pd.concat([dff["overall"], df_tfidf], axis=1)

In [32]:
print(df2.columns)

Index(['overall', '00', '000', '0005064716', '0060520620', '0060850523',
       '0060898526', '0060935502', '0061561622', '0061808407',
       ...
       'zone', 'zones', 'zoo', 'zoom', 'zooms', 'zoroastrianism', 'zucchini',
       'zuckoff', 'zufelt', 'zunis'],
      dtype='object', length=23572)


In [36]:
for i in df_tfidf.columns:
    print(i)

00
000
0005064716
0060520620
0060850523
0060898526
0060935502
0061561622
0061808407
0062024027
007
0142002070
014303653x
0153003693
0199535744
02
0261102664
0310718120
0312367546
0316769177
0349100135
0374530327
0375708448
0394726251
039480029x
0399501487
0439023521
0440412676
0451524934
048644287x
05
0525423648
0544336267
0553086766
0553575635
0553582445
0618101365
0646418432
0679728260
0679741801
0679745408
0711930015
0739378473
0741449102
0743244583
0743277708
0743477367
0747544115
0802811167
0805209999
0807275123
0812974263
0815410468
0971059829
10
100
1000
100s
101
101st
104
105
106
10mb
10th
10times
11
110
1105
115
117
1194
11th
12
121
122
123
128
12th
13
1300
1347
1349
1364
13th
14
1410
1415
1441746714
1442409053
1451673310
1456560832
149
14th
15
150
1500
1500s
152
1574
1577315855
1579126278
1579127371
1583
159
1592574165
1596
15am
15th
16
160
1600
1600s
1612931030
1621
1630
165
1650
1675
1687
16th
17
170
1700
1711
1724
1726
1741
1748
1765
1776
1781
1783
1793
1796
1799
17th
18
1

cruel
crueler
cruelest
cruelity
cruelly
cruelties
cruelty
cruise
cruising
crumbling
crump
crusade
crusading
crush
crushed
crushing
crusoe
crusty
crux
cry
crybaby
cryin
crying
cryptic
cryptomicron
cryptonomicon
crystal
crystalline
cs
csi
ct
cuba
cuban
cubano
cubicle
cuckold
cuddle
cue
cuenta
cufflinks
cuffs1
cuisine
cuisines
cul
culinary
culled
culloden
culminated
culminates
culminating
culprit
cult
cultic
cultish
cultivate
cults
cultural
culturally
culture
cultures
cum
cumberbatch
cumbersome
cumbrian
cumming
cumpari
cunning
cunningham
cup
cuppa
cups
curator
curb
curdie
cure
cured
cures
curiosity
curious
curiously
curl
curled
currant
currency
current
currentgift
currently
currents
curriculum
curried
curry
curse
curseand
cursed
curses
cursing
curtain
curve
curved
curves
curvy
curzon
cushman
cushy
cussing
cussler
custard
custody
custom
customary
customer
customers
customization
customs
cut
cute
cutest
cutesy
cutout
cuts
cutthroat
cutting
cyber
cycle
cycles
cyclical
cycling
cyclone
cyclops

historian
historians
historic
historical
historically
historicity
histories
history
historythe
hit
hitherto
hitler
hitlers
hitman
hits
hitting
hitty
hiughly
hiv
hive
hives
hizdahr
hmmmmmmm
hms
ho
hoarder
hoarding
hoax
hobb
hobbes
hobbies
hobbit
hobbits
hobbs
hobby
hobnailed
hockey
hodge
hodgepodge
hodkinson
hoenir
hoff
hoffnung
hofstadter
hofstede
hogan
hogwash
hokey
holbrook
hold
holden
holder
holders
holding
holds
hole
holes
holiday
holidays
holland
hollandaise
hollered
hollin
hollow
hollows
holly
hollywood
holman
holmes
holocaust
hologram
holographic
holwood
holy
homage
home
homebase
homegrown
homeland
homeless
homely
homemade
homeport
homer
homerian
homeric
homes
homeschool
homeschooling
homestead
hometown
hometowns
homework
homey
homicidal
homicide
homicides
homilies
homo
homoerotic
homogeneity
homophobic
homosexual
homosexuality
homosexualtiy
homossexual
hone
honed
hones
honest
honestly
honesty
honey
honeyman
honeymoon
honeysuckle
hong
honky
honor
honorable
honore
honored
honorif

photos
photoshop
phrase
phrased
phrases
phu
phycologically
physical
physically
physician
physicians
physiciansway
physicist
physicists
physics
physiology
pi
piano
piccolomini
pick
picked
pickering
picket
picking
pickings
pickle
picks
pickup
picky
picnic
pico
pictorial
pictsies
picture
picturebook
pictured
pictures
picturesque
picturing
piddle
pie
piebalds
piece
pieces
piera
pierce
pietism
pietists
piety
pig
pigafetta
pigeon
pigeonhole
pigments
pigs
pike
pilcher
pile
piled
piles
pilgrim
pilgrimages
pilgrims
piling
pill
pillage
pillaging
pillows
pills
pilot
piloted
pilots
pin
pinborough
pinch
pines
pink
pinnacle
pinned
pinot
piny
pioneer
pioneers
pious
piousness
pip
pipe
piper
pippin
pips
pique
piqued
pirate
pirates
pirot
pirsig
piscary
pitch
pitcher
pitchers
pitches
pitching
pitfalls
pithy
pitiable
pitied
pities
pitiless
pits
pitt
pitted
pitting
pittsburgh
pittu
pity
pitying
pius
pivotal
pixie
pixy
pizza
pkd
placate
place
placebo
placed
placeholder
placeholders
placemarker
placement
pla

swell
swelling
swells
swept
swerves
swest
swifly
swift
swim
swimmers
swimming
swimmingthe
swindled
swine
swing
swinging
swings
swiping
swirling
swiss
switch
switched
switches
switching
switzerland
swooped
sword
swordfight
swords
swordsman
swore
sworn
swpoa
sybil
sybs
sycophancy
syfy
syllable
syllables
sylvia
symbol
symbolic
symbolism
symbolize
symbolized
symbolizes
symbolizing
symbols
symetry
symmetry
sympathetic
sympathetically
sympathic
sympathize
sympathized
sympathizer
sympathizes
sympathy
symphony
symposium
symptoms
sync
synced
syndicate
syndrome
synergetics
synergy
synonymous
synonyms
synopses
synopsis
syntax
synthesis
synthesize
synthetic
syria
syringe
syrup
syrupy
sys
system
systematic
systemic
systemically
systems
szara
t3
t4
ta
tab
tabitha
table
tables
tablespoons
tablet
taboo
tabs
tacitly
taciturn
tacking
tackle
tackled
tackles
taco
tactics
tad
tadaaaa
tag
tahereh
tail
tailor
tails
taint
tainted
tak
takata
take
taken
takeoff
taker
takes
taking
tal
talanted
talbot
tale
talent